<a href="https://colab.research.google.com/github/Keerthu8999/GoogleColab/blob/main/MyFatePart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Attribute Extraction App

##  Part  1

Loading Dataseet

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip
!ls

In [2]:
import sys
sys.path.append("/content")
from CoNLL2Spacy import *

In [ ]:
# Test File
file = open("/content/Crimetest.txt", "r",encoding = "utf-8")
valList = []
for line in file:
    valList.append(line[:-1])
valList[:15]

In [4]:
TEST_DATA = conll2spacy(valList)

In [ ]:
# Train File
file = open("/content/Crimetrain.txt", "r",encoding = "utf-8")
trainList = []
for line in file:
    trainList.append(line[:-1])
trainList[:15]

In [ ]:
TRAIN_DATA = conll2spacy(trainList)
print(TRAIN_DATA[0])
print(type(TRAIN_DATA[0]))

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import spacy
import numpy
import pickle


In [14]:
import numpy as np
import spacy

# Define the path to the full word vector file
full_vectors_loc = "/content/wiki-news-300d-1M.vec"

# Define the path to the subset file
subset_vectors_loc = "/content/subset_vectors.vec"

# Define the size of the subset (e.g., 100,000)
subset_size = 500000

# Create a subset of word vectors
def create_subset(full_file, subset_file, subset_size):
    with open(full_file, "rb") as full_file:
        with open(subset_file, "wb") as subset_file:
            # Write the header
            header = full_file.readline()
            subset_file.write(header)

            # Write a subset of vectors
            for _ in range(subset_size):
                line = full_file.readline()
                subset_file.write(line)

# Create the subset file
create_subset(full_vectors_loc, subset_vectors_loc, subset_size)

# Load the subset of word vectors into SpaCy
lang = "en"
nlp = spacy.blank(lang)
with open(subset_vectors_loc, "rb") as file_:
    header = file_.readline()
    nr_row, nr_dim = header.split()
    nlp.vocab.reset_vectors(width=int(nr_dim))
    for line in file_:
        line = line.rstrip().decode("utf8")
        pieces = line.rsplit(" ", int(nr_dim))
        word = pieces[0]
        vector = np.asarray([float(v) for v in pieces[1:]], dtype="f")
        nlp.vocab.set_vector(word, vector)

file_path = "/content/subset_vectors.vec"

# Open the file and count the number of lines
with open(file_path, 'r') as file:
    num_lines = sum(1 for line in file)

print("Number of lines in", file_path, ":", num_lines)

Number of lines in /content/subset_vectors.vec : 500001


In [15]:
text = "I lost his doctor in patient yesterday"
doc = nlp(text)
print("similarity btw",doc[3] , "and", doc[5]," :-", doc[3].similarity(doc[5]))

similarity btw doctor and patient  :- 0.6411671042442322


# Part  2

Steps involved :-
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [16]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing
from spacy.util import minibatch, compounding


# For evaluateing the model from testing set
from spacy.training import *
from spacy.scorer import Scorer

from tqdm import tqdm


In [17]:
from spacy.training import Example
def echa(ner_model, testing_data):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        print(pred,annot)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

def evaluate(ner_model, testing_data):
  print(ner_model)
  testing_data = TEST_DATA
  ner_model = spacy.blank(lang)
  ner = nlp.create_pipe("ner")
  if 'ner' not in ner_model.pipe_names:
      ner_model.add_pipe("ner", last=True)

  ner_model.initialize()
  scorer = Scorer()
  examples = []
  for input_, annot in testing_data:
      doc = ner_model.make_doc(input_)

      # Convert annotations to spaCy format
      entities = []
      for start, end, label in annot['entities']:
          entities.append((start, end, label))
      gold_entities = {"entities": entities}

      # Create Example object
      example = Example.from_dict(doc, gold_entities)

      # Apply ner_model to the Doc
      predicted_doc = ner_model(doc)
      example.predicted = predicted_doc
      examples.append(example)

  print(scorer.score(examples))
  return scorer.score(examples)

In [18]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

In [19]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [ ]:
from spacy.training import Example
f1score = 0.0000
modelName = "modelTrained"
lang="en"
vectors_loc = "/content/subset_vectors.vec"
modiner = spacy.blank(lang)
with open(vectors_loc, "rb") as file_:
    header = file_.readline()
    nr_row, nr_dim = header.split()
    modiner.vocab.reset_vectors(width=int(nr_dim))
    for line in file_:
        line = line.rstrip().decode("utf8")
        pieces = line.rsplit(" ", int(nr_dim))
        word = pieces[0]
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")
        modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab
if 'ner' not in modiner.pipe_names:
      ner = modiner.create_pipe('ner')
      modiner.add_pipe('ner', last=True)
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

examples = []

for text, annotations in TRAIN_DATA:
    doc = nlp(text)
    examples.append(Example.from_dict(doc, annotations))
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with modiner.disable_pipes(*other_pipes):  # only train NER
    optimizer = modiner.begin_training()
    for itn in range(3):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for example in tqdm(examples):
            modiner.update(
                [example],
                drop=0.5,
                losses=losses)
        print(TRAIN_DATA)
        print(losses)
        results = echa(modiner, TEST_DATA)
        print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
        #print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
        if f1score == 0.00:
            try:
                pnlp = spacy.load(modelName)
                result = echa(pnlp, TEST_DATA) # calling evaluate function
                f1score = result["ents_f"]
            except:
                print("Previous Model not found")

        print("Best Sccore :- ",f1score)
        print("------------------------------------")
        if f1score <= results["ents_f"]:
            f1score = results["ents_f"]
            modiner.to_disk(modelName)

    print("-----Best Model is Saved-----")

  6%|▋         | 2277/36418 [01:23<16:07, 35.30it/s]

In [ ]:
# traning the model with 100 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 1,droprate = 0.55, modelName = "CrimeNER")

In [ ]:
# loading the saved model
pnlp = loadNERModel("CrimeNER")

In [ ]:
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 86.47551444580847
Precision of Model is :- 87.0393931979447
Recall of Model is :- 85.91889476607976


In [ ]:
from spacy import displacy


In [ ]:
testcase = pnlp('he was killed due to harassment')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A woman was assault by two men near Shahdara')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('My name is Pooja I was near Domino-pizza in Pinjore a man on bike snatched my purse')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('My name is Amit I saw a woman killing a dog on 15th july near Kalujhanda')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A car hit me at Mansarovar Park at 5pm wraped')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('Yesterday there was incident of theft near my house')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('A man slapped a girl in the Sadar market')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

C:\Users\Shreya\anaconda3\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
testcase = pnlp('  ')
displacy.render(testcase, style='ent', jupyter=True)